### Libraries

In [1]:
#!pip install -U scikit-learn
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import f1_score,confusion_matrix

### Reading Dataset

In [2]:
df = pd.read_csv("data/wisconsin.csv", )
df.shape

(569, 32)

In [3]:
#df

### Data Analysis

In [4]:
#df.columns

In [5]:
#df.info()

In [6]:
#df.describe()

In [7]:
df.isnull().sum().sum()

0

In [8]:
df["diagnosis"].value_counts()

B    357
M    212
Name: diagnosis, dtype: int64

In [9]:
df = df.drop(columns=["id"])

### Feature Engineering

In [10]:
# to-do

In [11]:
label = "diagnosis"

In [12]:
def min_max_scale(column):
    if column.name == label:
        return column
    else:
        old_values = column.values.reshape(-1, 1)
        new_values = MinMaxScaler().fit_transform(old_values)
        new_values = new_values.reshape(-1)
        return new_values

In [13]:
df = df.apply(min_max_scale)

df.shape

(569, 31)

In [14]:
df

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,0.521037,0.022658,0.545989,0.363733,0.593753,0.792037,0.703140,0.731113,0.686364,...,0.620776,0.141525,0.668310,0.450698,0.601136,0.619292,0.568610,0.912027,0.598462,0.418864
1,M,0.643144,0.272574,0.615783,0.501591,0.289880,0.181768,0.203608,0.348757,0.379798,...,0.606901,0.303571,0.539818,0.435214,0.347553,0.154563,0.192971,0.639175,0.233590,0.222878
2,M,0.601496,0.390260,0.595743,0.449417,0.514309,0.431017,0.462512,0.635686,0.509596,...,0.556386,0.360075,0.508442,0.374508,0.483590,0.385375,0.359744,0.835052,0.403706,0.213433
3,M,0.210090,0.360839,0.233501,0.102906,0.811321,0.811361,0.565604,0.522863,0.776263,...,0.248310,0.385928,0.241347,0.094008,0.915472,0.814012,0.548642,0.884880,1.000000,0.773711
4,M,0.629893,0.156578,0.630986,0.489290,0.430351,0.347893,0.463918,0.518390,0.378283,...,0.519744,0.123934,0.506948,0.341575,0.437364,0.172415,0.319489,0.558419,0.157500,0.142595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,M,0.690000,0.428813,0.678668,0.566490,0.526948,0.296055,0.571462,0.690358,0.336364,...,0.623266,0.383262,0.576174,0.452664,0.461137,0.178527,0.328035,0.761512,0.097575,0.105667
565,M,0.622320,0.626987,0.604036,0.474019,0.407782,0.257714,0.337395,0.486630,0.349495,...,0.560655,0.699094,0.520892,0.379915,0.300007,0.159997,0.256789,0.559450,0.198502,0.074315
566,M,0.455251,0.621238,0.445788,0.303118,0.288165,0.254340,0.216753,0.263519,0.267677,...,0.393099,0.589019,0.379949,0.230731,0.282177,0.273705,0.271805,0.487285,0.128721,0.151909
567,M,0.644564,0.663510,0.665538,0.475716,0.588336,0.790197,0.823336,0.755467,0.675253,...,0.633582,0.730277,0.668310,0.402035,0.619626,0.815758,0.749760,0.910653,0.497142,0.452315


### 1) Feature selection with correlation

In [15]:
drop_list1 = ['perimeter_mean','radius_mean','compactness_mean','concave points_mean','radius_se','perimeter_se','radius_worst','perimeter_worst','compactness_worst','concave points_worst','compactness_se','concave points_se','texture_worst','area_worst']
df = df.drop(drop_list1,axis = 1 )        # do not modify x, we will use it later 
df.head()

,diagnosis,texture_mean,area_mean,smoothness_mean,concavity_mean,symmetry_mean,fractal_dimension_mean,texture_se,area_se,smoothness_se,concavity_se,symmetry_se,fractal_dimension_se,smoothness_worst,concavity_worst,symmetry_worst,fractal_dimension_worst
0,M,0.022658,0.363733,0.593753,0.703140,0.686364,0.605518,0.120469,0.273811,0.159296,0.135682,0.311645,0.183042,0.601136,0.568610,0.598462,0.418864
1,M,0.272574,0.501591,0.289880,0.203608,0.379798,0.141323,0.082589,0.125660,0.119387,0.046970,0.084539,0.091110,0.347553,0.192971,0.233590,0.222878
2,M,0.390260,0.449417,0.514309,0.462512,0.509596,0.211247,0.094303,0.162922,0.150831,0.096768,0.205690,0.127006,0.483590,0.359744,0.403706,0.213433
3,M,0.360839,0.102906,0.811321,0.565604,0.776263,1.000000,0.175875,0.038155,0.251453,0.142955,0.728148,0.287205,0.915472,0.548642,1.000000,0.773711
4,M,0.156578,0.489290,0.430351,0.463918,0.378283,0.186816,0.093065,0.163688,0.332359,0.143636,0.136179,0.145800,0.437364,0.319489,0.157500,0.142595


#### Dataset split

In [16]:
df_train, df_test = train_test_split(df, test_size=0.3, stratify=df[label], random_state=42)

In [17]:
df_train.shape

(398, 17)

In [18]:
df_test.shape

(171, 17)

#### Features-Label split

In [46]:
X_train = df_train.loc[:, df_train.columns != label]
y_train = df_train.loc[:, df_train.columns == label]

X_test = df_test.loc[:, df_test.columns != label]
y_test = df_test.loc[:, df_test.columns == label]

X_train.shape

(398, 16)

In [47]:
X_train.head()

,texture_mean,area_mean,smoothness_mean,concavity_mean,symmetry_mean,fractal_dimension_mean,texture_se,area_se,smoothness_se,concavity_se,symmetry_se,fractal_dimension_se,smoothness_worst,concavity_worst,symmetry_worst,fractal_dimension_worst
78,0.482246,0.467232,0.685836,0.879569,0.932323,0.662595,0.336987,0.204704,0.294626,0.275505,1.000000,0.175926,0.612362,0.613498,0.763848,0.292536
330,0.196145,0.275589,0.381692,0.282099,0.364646,0.206403,0.085617,0.049436,0.140259,0.093813,0.095514,0.076911,0.477646,0.395847,0.286615,0.237439
378,0.183970,0.185408,0.271283,0.099555,0.369697,0.189764,0.040112,0.008495,0.118945,0.061692,0.144903,0.137266,0.371987,0.205192,0.359156,0.271153
213,0.536016,0.341251,0.433059,0.394096,0.125253,0.183235,0.288808,0.096616,1.000000,0.363131,0.195137,0.403010,0.350855,0.223882,0.007491,0.086187
89,0.187014,0.215652,0.546809,0.233505,0.533333,0.284330,0.083319,0.067161,0.129007,0.112020,0.230596,0.136333,0.373308,0.207987,0.312636,0.194740


### 2) Univariate feature selection

In [48]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
# find best scored 5 features
select_feature = SelectKBest(chi2, k=5).fit(X_train, y_train)
print('Score list:', select_feature.scores_)
print('Feature list:', X_train.columns)

Score list: [3.84538882e+00 2.03657696e+01 2.53775484e+00 3.06196825e+01
 1.97970437e+00 3.66545637e-03 1.73972635e-02 1.35240345e+01
 5.85621267e-02 1.31403514e+00 7.91760460e-05 4.95843510e-02
 4.63671197e+00 2.01574199e+01 4.17854903e+00 2.62786200e+00]
Feature list: Index(['texture_mean', 'area_mean', 'smoothness_mean', 'concavity_mean',
       'symmetry_mean', 'fractal_dimension_mean', 'texture_se', 'area_se',
       'smoothness_se', 'concavity_se', 'symmetry_se', 'fractal_dimension_se',
       'smoothness_worst', 'concavity_worst', 'symmetry_worst',
       'fractal_dimension_worst'],
      dtype='object')


In [49]:
X_train = select_feature.transform(X_train)
X_test = select_feature.transform(X_test)
pd.DataFrame(X_train.columns)
#.cv_results_).sort_values("rank_test_score")
#print('Feature list:', X_train.columns)

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

### 3) Feature Extraction with PCA

In [23]:
#from sklearn.decomposition import PCA
#pca = PCA()
#pca.fit(X_train)

PCA()

### Modelling

#### 1. SVC

#### Object

In [24]:
svc_obj = SVC()

#### Params

In [25]:
svc_params = {
    'kernel':('linear', 'rbf', 'sigmoid'),
    'C':[i for i in range(1, 11)]
}

#### Create a grid search object and train it on all combinations

In [26]:
svc_gs_clf = GridSearchCV(svc_obj, svc_params)

svc_gs_clf.fit(X_train, y_train.values.reshape(-1))

GridSearchCV(estimator=SVC(),
             param_grid={'C': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'kernel': ('linear', 'rbf', 'sigmoid')})

#### Get best parameters as per grid search

In [27]:
svc_gs_clf.best_estimator_

SVC(C=8, kernel='linear')

#### Get the CV score on best parameters

In [28]:
svc_gs_clf.best_score_

0.9446518987341772

### Display all grid search results

In [29]:
#pd.DataFrame(svc_gs_clf.cv_results_).sort_values("rank_test_score")

#### Calculate test scores on best parameters

In [30]:
svc_gs_clf.score(X_test, y_test.values.reshape(-1))

0.9532163742690059

#### 2. Decision Tree

In [31]:
dt_obj = DecisionTreeClassifier()

In [32]:
dt_params = {
    "max_depth": [i for i in range(3, 11)],
    "min_samples_split": [i for i in range(2, 5)],
    "min_samples_leaf": [i for i in range(1, 5)]
}

In [33]:
dt_gs_clf = GridSearchCV(dt_obj, dt_params)

dt_gs_clf.fit(X_train, y_train.values.reshape(-1))

GridSearchCV(estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': [3, 4, 5, 6, 7, 8, 9, 10],
                         'min_samples_leaf': [1, 2, 3, 4],
                         'min_samples_split': [2, 3, 4]})

In [34]:
dt_gs_clf.best_estimator_

DecisionTreeClassifier(max_depth=7, min_samples_split=3)

In [35]:
dt_gs_clf.best_score_

0.959746835443038

In [36]:
#pd.DataFrame(dt_gs_clf.cv_results_).sort_values("rank_test_score")

In [37]:
dt_gs_clf.score(X_test, y_test.values.reshape(-1))

0.9064327485380117

#### 3. Gaussian NB

In [38]:
gnb_clf =  GaussianNB()

gnb_clf.fit(X_train, y_train.values.reshape(-1))

gnb_clf.score(X_test, y_test.values.reshape(-1))

0.935672514619883

 #### 4.RandomForest 

In [39]:
#rf_clf = RandomForestClassifier(random_state=43 , )      

#rf_clf.fit(X_train,y_train.values.reshape(-1))

#rf_clf.score(X_test, y_test.values.reshape(-1))

In [40]:
rf_obj = RandomForestClassifier()
rf_params = {
    'criterion':('gini', 'entropy'),
    'n_estimators':[i for i in range(100, 105)],
    'max_features' : ( 'auto', 'sqrt', 'log2')
}

In [41]:
rf_gs_clf = GridSearchCV(rf_obj, rf_params)

rf_gs_clf.fit(X_train, y_train.values.reshape(-1))

GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'criterion': ('gini', 'entropy'),
                         'max_features': ('auto', 'sqrt', 'log2'),
                         'n_estimators': [100, 101, 102, 103, 104]})

In [42]:
svc_gs_clf.score(X_test, y_test.values.reshape(-1))

0.9532163742690059

In [43]:
#ac = accuracy_score(y_test,rf_clf.predict(X_test))
#print('Accuracy is: ',ac)